In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,Input,Flatten
from tensorflow.keras.models import Model
import os
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train=pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
test=pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['label'].hist()

In [ ]:
def get_image(image_path):
    img=load_img(image_path,target_size=(256,256,3))
    img=img_to_array(img)
    img=img/255.
    
    return img

In [ ]:
train_path=[os.path.join('../input/cassava-leaf-disease-classification/train_images',image_name) for image_name in  train['image_id']]

In [ ]:
def gen(train_labels,train_path,batch_size=32):
    images=[]
    labels=[]
    i=0
    b=0
    while True:
        images.append(get_image(train_path[i]))
        labels.append(train_labels[i])
        b+=1
        i+=1
        
        if b==batch_size:
            yield np.array(images),np.array(tf.keras.utils.to_categorical(labels,5))
            b=0
def valid_gen(train_labels,train_path,batch_size=32):
    images=[]
    labels=[]
    i=0
    b=0
    while True:
        images.append(get_image(train_path[i]))
        labels.append(train_labels[i])
        b+=1
        i+=1
        
        if b==batch_size:
            yield np.array(images),np.array(tf.keras.utils.to_categorical(labels,5))
            b=0

In [ ]:
len(train)

In [ ]:
train['label']=train['label'].astype(str)

In [ ]:
train_gen = ImageDataGenerator(
zoom_range=[0.5,0.2],
brightness_range=[0.5,1.0],
rotation_range=90,
horizontal_flip=True,
vertical_flip=True,
height_shift_range=0.2,
width_shift_range=0.2
)

datagen = train_gen.flow_from_dataframe(train[:20000],
                                        directory="../input/cassava-leaf-disease-classification/train_images",
                                       x_col="image_id",
                                       y_col="label",
                                       batch_size=32)
val_gen = ImageDataGenerator()
val_data = val_gen.flow_from_dataframe(train[20000:],
                                        directory="../input/cassava-leaf-disease-classification/train_images",
                                       x_col="image_id",
                                       y_col="label",
                                       batch_size=1)

In [ ]:
for i in range(9):
    plt.subplot(330+1+i)
    batch=datagen.next()
    image=batch[0].astype(np.uint8)
    plt.imshow(image[0])
#    print(image.shape)

In [ ]:
from tensorflow.keras.applications import MobileNet,MobileNetV2,ResNet50,VGG16

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight="balanced",classes=train['label'].unique(),y = train['label'])

In [ ]:
input_image=Input((256,256,3))

mobilenet=MobileNet(include_top=False,weights="../input/keras-pretrained-imagenet-weights/mobilenet_imagenet_1000_no_top.h5")
x=mobilenet(input_image)
x=Flatten()(x)
x=Dense(128,activation='relu')(x)
output=Dense(5,activation='softmax')(x)

model_1=Model(input_image,output)
print(model_1.summary())
model_1.layers[1].trainable=False
print(model_1.summary())

input_image_2=Input((256,256,3))

mobilenetv2=MobileNetV2(include_top=False,weights="../input/keras-pretrained-imagenet-weights/mobilenetv2_imagenet_1000_no_top.h5")
x=mobilenetv2(input_image_2)
x=Flatten()(x)
x=Dense(128,activation='relu')(x)
output=Dense(5,activation='softmax')(x)

model_2=Model(input_image_2,output)
model_2.layers[1].trainable=False


# input_image_3=Input((256,256,3))

# resnet=ResNet50(include_top=False,weights="https://github.com/fchollet/deep-learning-models/releases/download/v0.1/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5",input_shape=(256,256,3))
# x=resnet(input_image_3)
# x=Flatten()(x)
# x=Dense(128,activation='relu')(x)
# output=Dense(5,activation='softmax')(x)

# model_3=Model(input_image_3,output)


input_image_4=Input((256,256,3))

vgg16=VGG16(include_top=False,weights="../input/keras-pretrained-imagenet-weights/vgg16_imagenet_1000_no_top.h5")
x=vgg16(input_image_4)
x=Flatten()(x)
x=Dense(128,activation='relu')(x)
output=Dense(5,activation='softmax')(x)

model_4=Model(input_image_4,output)
model_2.layers[1].trainable=False

In [ ]:
model_1.compile(loss="categorical_crossentropy",metrics="accuracy",optimizer="adam")
model_2.compile(loss="categorical_crossentropy",metrics="accuracy",optimizer="adam")
model_4.compile(loss="categorical_crossentropy",metrics="accuracy",optimizer="adam")

In [ ]:
class_weights

In [ ]:
class_weight={}
for i,j in enumerate(class_weights):
    class_weight[i]=j

In [ ]:
model_1.fit_generator(datagen,steps_per_epoch=100,epochs=10,validation_data=val_data,class_weight=class_weight)

In [ ]:
model_2.fit_generator(datagen,steps_per_epoch=100,epochs=10,validation_data=val_data,class_weight=class_weight)

In [ ]:
model_4.fit_generator(datagen,steps_per_epoch=100,epochs=10,validation_data=val_data,class_weight=class_weight)

In [ ]:
test['label']=(np.argmax(model_1.predict(np.array([get_image('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')])))+np.argmax(model_2.predict(np.array([get_image('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')])))+np.argmax(model_4.predict(np.array([get_image('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')]))))//3


In [ ]:
test.to_csv('submission.csv',index=False)

In [ ]:
!ls ./

In [ ]:
#from sklearn.ensemble import RandomForestClassifier

In [ ]:
# y_val=[]
# x_meta=[]
# i=0
# while True:
#     batch = val_data.next()
#     image = batch[0].astype(np.uint8)
#     label = np.argmax(batch[1])
#     y_val.append(label)
#     x=[]
#     for model in [model_1,model_2,model_4]:
#         x.append(np.argmax(model.predict(image)))
#     x_meta.append(x)
#     #print(label)
#     i+=1
#     if i>=1397:
#         break

In [ ]:
# x_meta

In [ ]:
# input_image=Input((64,63,3))

# conv1=Conv2D(32,(3,3),activation='relu')(input_image)
# conv1=MaxPooling2D((2,2))(conv1)

# conv1=Conv2D(64,(3,3),activation='relu')(conv1)
# conv1=MaxPooling2D((2,2))(conv1)
# conv1=Dropout(0.2)(conv1)

# conv1=Conv2D(128,(3,3),activation='relu')(conv1)
# conv1=MaxPooling2D((2,2))(conv1)
# conv1=Dropout(0.5)(conv1)

# conv1=Conv2D(256,(3,3),activation='relu')(conv1)
# conv1=MaxPooling2D((2,2))(conv1)

# flat=Flatten()(conv1)
# dense1=Dense(256,activation='relu')(flat)
# dense1=Dropout(0.4)(dense1)

# dense2=Dense(64,activation='relu')(dense1)
# dense_out=Dense(5,activation='softmax')(dense2)

# model=Model(input_image,dense_out)
# model.summary()

In [ ]:
# model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [ ]:
# model.fit_generator(gen(train['label'][:19000].values,train_path[:20000]),steps_per_epoch=32//5,epochs=10,validation_data=valid_gen(train['label'][20000:20100].values,train_path[20000:20100],1))

In [ ]:
# model.history.history.keys()

In [ ]:
# test['label']=np.argmax(model.predict(np.array([get_image('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')])))


In [ ]:
#test.to_csv('submission.csv',index=False)